In [9]:
#Глобальный импорт
import re
from random import randint
import uuid
import os
from itertools import islice

In [106]:
# глобальные переменные
n = 1000

# Создаем синтетические данные на основе единой строки
## предполагаем, что формат строки не меняется

In [2]:
# генерируем пять рандомных названий файлов с расширением logs
file_names = [str(uuid.uuid4()) + ".logs" for x in range(5)]

In [80]:
# функция генерирует строку
def generate_string():
    action_id_index = randint(0,10)
    action_number = randint(90,101)
    user_id_index = randint(0,10)
    # uid пользователья(в реальной жизни - это юзер id который нужно отлавить)
    user_uid = '"' + str(uuid.uuid4()) + '"'
    
    original_str:str = """Oct 29 18:10:43 ds-db27-prod1 journal: Oracle Audit[13491]: LENGTH: "433" SESSIONID:[9] "203830992" ENTRYID:[1] "1" STATEMENT:[1] "1" USERID:[14] "ZABBIX_MONITOR" USERHOST:[31] "vm-pr-infra-zabbix01.tcsbank.ru" ACTION:[{}] "{}" RETURNCODE:[1] "0" COMMENT$TEXT:[138] "Authenticated by: DATABASE;AUTHENTICATED IDENTITY: ZABBIX_MONITOR; Client address: (ADDRESS=(PROTOCOL=tcp)(HOST=10.217.12.96)(PORT=42594))" OS$USERID:[{}] {} DBID:[10] "1033166545" PRIV$USED:[1] "5" CURRENT_USER:[14] "ZABBIX_MONITOR""".format(action_id_index, action_number, user_id_index, user_uid)
    original_str
    return original_str
def get_user_id(lg_str:str):
    return re.findall('"([^"]*)"', lg_str.split("USERID")[1].split("USERHOST")[0])[0]
def get_os_user_id(lg_str:str):
    return re.findall('"([^"]*)"', lg_str.split("OS$USERID")[1].split("DBID")[0])[0]
def log_event(lg_str: str):
    return re.findall('"([^"]*)"', lg_str.split("ACTION")[1].split("RETURNCODE")[0])[0]
def get_sension_id(lg_str: str):
    return re.findall('"([^"]*)"', lg_str.split("SESSIONID")[1].split("ENTRYID")[0])[0]

In [4]:
# создаем и пишем в файлы синтетику
for name_file in file_names:
    with open(name_file, "w") as f:
        for i in range(10000):
            f.writelines(generate_string() + "\n")

# проверка работы ф-ций

In [17]:
# генерируем строку
generate_string()

'Oct 29 18:10:43 ds-db27-prod1 journal: Oracle Audit[13491]: LENGTH: "433" SESSIONID:[9] "203830992" ENTRYID:[1] "1" STATEMENT:[1] "1" USERID:[14] "ZABBIX_MONITOR" USERHOST:[31] "vm-pr-infra-zabbix01.tcsbank.ru" ACTION:[2] "91" RETURNCODE:[1] "0" COMMENT$TEXT:[138] "Authenticated by: DATABASE;AUTHENTICATED IDENTITY: ZABBIX_MONITOR; Client address: (ADDRESS=(PROTOCOL=tcp)(HOST=10.217.12.96)(PORT=42594))" OS$USERID:[7] "548e9ad7-c293-4e7d-8c66-55cc907efa09" DBID:[10] "1033166545" PRIV$USED:[1] "5" CURRENT_USER:[14] "ZABBIX_MONITOR'

In [18]:
# достаем os_user_id
get_os_user_id(generate_string())

'3c23683f-9b4c-49bf-8490-c6ee7d9b9020'

In [20]:
# достаем user_id
get_user_id(generate_string())

'ZABBIX_MONITOR'

In [22]:
# достаем событие
log_event(generate_string())

'94'

# Сам скрипт

## 1. есть входной массив учетных записей -> вывести строку из логов где встречается эта учетка -> удалить из массива эту учетку

In [131]:
file_name_result_chapter_one = "result_chapter_one.txt"
file_errors_one = "errors_one.txt"

In [132]:
# создаем массив с учетками, которые нужно найти
# ДОБАВИТЬ СЮДА ЧЕРЕЗ ЗАПЯТУЮ недостающие учетки
user_ids = ["ZABBIX_MONITOR"]

In [133]:
arr_line_logs_result:list = []
arr_errors_one:list = []
i = 0
for file in os.listdir():
    if "log" in file:
        with open(file) as f:
            while True:
                next_n_lines = list(islice(f, n))
                if not next_n_lines:
                    break
                else:
                    # тут процессим наши строки
                    for lines in next_n_lines:
                        i = i + 1
                        try:
                            user_id = get_user_id(lines)
                            # реализация бизнес логики
                            if user_id in user_ids and len(user_ids) > 0:
                                arr_line_logs_result.append(lines)
                                user_ids.remove(user_id)
                        except:
                            arr_errors_one.append(lines)

In [135]:
print("успешно оббработанных строк:{}".format(i))
print("неуспешных:{}".format(len(arr_errors_one)))

успешно оббработанных строк:110000
неуспешных:0


In [120]:
with open(file_name_result_chapter_one, "w") as f:
    f.writelines(arr_line_logs_result)

In [ ]:
with open(file_errors_one, "w") as f:
    f.writelines(arr_errors_one)

## 2. два условия дя поиска:  action 100 and одна из учеток в массиве

In [139]:
file_name_result_chapter_two = "result_chapter_two.txt"
file_errors_two = "errors_two.txt"
user_ids = ["ZABBIX_MONITOR"]

In [142]:
arr_os_user_id_cond:list = []
arr_errors_two:list = []
j = 0
for file in os.listdir():
    if "log" in file:
        with open(file) as f:
            # читаем чанками файлы чтобы не использовать много оперативной памяти
            while True:
                next_n_lines = list(islice(f, n))
                if not next_n_lines:
                    break
                else:
                    # тут процессим наши строки
                    for lines in next_n_lines:
                        j = j + 1
                        try:
                            if log_event(lines) == "100" and get_user_id(lines) in user_ids:
                                # тут бизнес смысл
                                arr_os_user_id_cond.append("файл:" + file + "|  "+"ид сессии:"+ get_sension_id(lines) +"|  "+"os_user_id:"+ get_os_user_id(lines) + "\n")
                        except:
                            arr_errors_two.append(lines)

In [143]:
print("успешно оббработанных строк:{}".format(j))
print("неуспешных:{}".format(len(arr_errors_two)))

успешно оббработанных строк:110000
неуспешных:0


In [144]:
# пишем результат второй задачи

In [128]:
with open(file_name_result_chapter_two, "w") as f:
    f.writelines(arr_os_user_id_cond)

In [ ]:
with open(file_errors_two, "w") as f:
    f.writelines(arr_errors_two)